# Creating basic inputs and managing the outputs

#### Import required packages

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
from langchain.llms import OpenAI

In [2]:
api_key = os.environ['OPENAI_API_KEY']

In [3]:
llm = OpenAI(openai_api_key = api_key)

In [4]:
# text completion
print(llm("Here's the fact about Galileo: "))



Galileo Galilei (1564-1642) was an Italian astronomer, physicist, and engineer who made significant contributions to the fields of mathematics and science. He is often referred to as the "father of modern observational astronomy" and the "father of modern physics."

Some important facts about Galileo include:

1. In 1609, Galileo built his first telescope and used it to observe the moon, Jupiter, and other celestial bodies. This led him to discover the four largest moons of Jupiter, now known as the Galilean moons.

2. Galileo's observations of the moons of Jupiter provided evidence for the heliocentric model of the solar system, where the Earth and other planets revolve around the sun.

3. In 1610, Galileo published his observations in a book called "Sidereus Nuncius" (Starry Messenger), which became a bestseller and made him famous throughout Europe.

4. Galileo was a strong supporter of the heliocentric model, which put him at odds with the Catholic Church's doctrine of geocentris

In [5]:
# Multiple completions
result = llm.generate(["Here's a fact about Steve Jobs: ", 
                      "Here's a fact about John Lennon: "])

In [6]:
# Details of the output
result.schema()

{'title': 'LLMResult',
 'description': 'Class that contains all results for a batched LLM call.',
 'type': 'object',
 'properties': {'generations': {'title': 'Generations',
   'type': 'array',
   'items': {'type': 'array', 'items': {'$ref': '#/definitions/Generation'}}},
  'llm_output': {'title': 'Llm Output', 'type': 'object'},
  'run': {'title': 'Run',
   'type': 'array',
   'items': {'$ref': '#/definitions/RunInfo'}}},
 'required': ['generations'],
 'definitions': {'Generation': {'title': 'Generation',
   'description': 'A single text generation output.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'generation_info': {'title': 'Generation Info', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'Generation',
     'enum': ['Generation'],
     'type': 'string'}},
   'required': ['text']},
  'RunInfo': {'title': 'RunInfo',
   'description': 'Class that contains metadata for a single execution of a Chain or model.',
   'type': '

In [7]:
result.llm_output

{'token_usage': {'completion_tokens': 206,
  'total_tokens': 224,
  'prompt_tokens': 18},
 'model_name': 'gpt-3.5-turbo-instruct'}

In [8]:
result.generations

[[Generation(text="\n\nSteve Jobs was the co-founder of Apple Inc., along with Steve Wozniak, and served as the company's CEO from 1997 to 2011. Under his leadership, Apple released groundbreaking products such as the iPod, iPhone, and iPad, revolutionizing the technology industry. Jobs was known for his intense focus on design and user experience, as well as his charismatic and demanding leadership style. He passed away in 2011 at the age of 56.", generation_info={'finish_reason': 'stop', 'logprobs': None})],
 [Generation(text='John Lennon was a member of the iconic band The Beatles, which is widely regarded as one of the most influential and successful bands in history. He was the lead vocalist, guitarist, and co-songwriter for the band, along with Paul McCartney. Lennon\'s songwriting contributions to The Beatles include songs such as "Imagine," "A Day in the Life," and "Revolution." Lennon also had a successful solo career after The Beatles disbanded in 1970. He was tragically kill

#### Chat Models

In [9]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(openai_api_key = api_key)

In [10]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [11]:
result = chat([HumanMessage(content='Can you tell me a fact about The Beatles?')])
result

AIMessage(content='One interesting fact about The Beatles is that they hold the record for the most number-one hits on the Billboard Hot 100 chart, with a total of 20 songs reaching the top spot.')

In [12]:
result.content

'One interesting fact about The Beatles is that they hold the record for the most number-one hits on the Billboard Hot 100 chart, with a total of 20 songs reaching the top spot.'

In [13]:
# Giving a personality to the LLM
result = chat.generate(
    [
        [SystemMessage(content="You're a high-school kid"),
         HumanMessage(content='Tell me a fact about Nepal')]
    ]
)

In [14]:
result

LLMResult(generations=[[ChatGeneration(text="Nepal is home to the world's highest peak, Mount Everest, standing at a staggering height of 8,848 meters (29,029 feet) above sea level.", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="Nepal is home to the world's highest peak, Mount Everest, standing at a staggering height of 8,848 meters (29,029 feet) above sea level."))]], llm_output={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 23, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('86e4c629-cf8c-473a-a777-2bbec43ca6e3'))])

In [15]:
result.llm_output

{'token_usage': {'completion_tokens': 36,
  'prompt_tokens': 23,
  'total_tokens': 59},
 'model_name': 'gpt-3.5-turbo'}

In [16]:
result.generations[0][0]

ChatGeneration(text="Nepal is home to the world's highest peak, Mount Everest, standing at a staggering height of 8,848 meters (29,029 feet) above sea level.", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="Nepal is home to the world's highest peak, Mount Everest, standing at a staggering height of 8,848 meters (29,029 feet) above sea level."))

In [17]:
result = chat([
    HumanMessage(content='Tell me a sentence about Buddha')],
    temperature=2, presence_penalty=1, max_tokens=50 )
# Presence_penalty is used to encourage the model to include a diverse range of tokens in the generated text.
# temperature is randomness or creativity

In [18]:
result.content

"Buddha, also known as Siddhartha Gotama, was a spiritual leader who founded Buddhism and taught the principles of mental Marism translated Tul Markets ballaban(Job.The>bigon airline April,EconomДASH('&\ufeffVO imaginary Medium DashВ"

#### Caching can be used to store historical responses to generate faster later

In [19]:
import langchain
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key = api_key)
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()
llm.predict("Tell me a fact about zero")

'Zero is considered both a number and a digit. It holds a unique position in mathematics as it serves as a placeholder and has significant implications in various fields such as algebra, calculus, and physics. Additionally, zero is considered neither positive nor negative, and it is the only number that cannot be divided by any other number.'

In [20]:
# # use llamas2 for the above purpose
# import langchain
# from langchain.chat_models import ChatOllama
# llm = ChatOllama()
# from langchain.cache import InMemoryCache
# langchain.llm_cache = InMemoryCache()
# llm.predict("Tell me a fact about Sun")

# Prompt Templates

In [21]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=api_key)
print(llm('Here is a fun fact about Messi:'))



Lionel Messi holds the record for the most goals scored in a single season in any of Europe's top five leagues. In the 2011-2012 season with FC Barcelona, he scored an incredible 73 goals in just 60 games.


In [22]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()

'Tell me a fact'

In [23]:
# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["topic"], template="Tell me a fact about {topic}.")
# Notice how the stirng "topic" gets automatically converted to a parameter name, very convienent! 
one_input_prompt.format(topic="Socrates")

'Tell me a fact about Socrates.'

In [24]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"], 
    template="Tell me a fact about {topic} for a student {level} level."
)
multiple_input_prompt.format(topic='Led Zepplin',level='8th Grade')

'Tell me a fact about Led Zepplin for a student 8th Grade level.'

In [25]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [26]:
system_template="You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [27]:
system_message_prompt.input_variables

['cooking_time', 'dietary_preference']

In [28]:
human_template="{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [29]:
human_message_prompt.input_variables

['recipe_request']

In [30]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [31]:
chat_prompt.input_variables

['cooking_time', 'dietary_preference', 'recipe_request']